In [1]:
datafile = r'C:\Rahul\encyclopedia-titanica.html'

In [2]:
from bs4 import BeautifulSoup
with open(datafile,"r") as f:
    soup = BeautifulSoup(f,"html.parser")

In [3]:
table = soup.find('table')


In [4]:
import pandas as pd
import html5lib
data = pd.read_html(str(table),flavor='bs4')[0]

In [5]:
data.head()

,Name,Age,Class/Dept,Ticket,Joined,Job,Boat [Body],Unnamed: 7
0,"ABÄ« SA'B, Mr Jirjis YÅ«suf",26,3rd Class Passenger,2685Â£7 4s 6d,Cherbourg,Shoemaker Â,Â [312],NaN
1,"ABÄ«-AL-MUNÃ , Mr NÄ�sÄ«f QÄ�sim",27,3rd Class Passenger,2699Â£18 15s 9d,Cherbourg,Â,15Â,NaN
2,"ABBING, Mr Anthony",42,3rd Class Passenger,5547Â£7 11s,Southampton,Blacksmith Â,Â Â,NaN
3,"ABBOTT, Mrs Rhoda Mary 'Rosa'",39,3rd Class Passenger,CA2673Â£20 5s,Southampton,Â,AÂ,NaN
4,"ABBOTT, Mr Rossmore Edward",16,3rd Class Passenger,CA2673Â£20 5s,Southampton,Jeweller Â,Â [190],NaN


In [6]:
def cleanup(value):
    return value.encode('ascii',errors='replace').replace("?", " ")

data['Name']=data['Name'].apply(cleanup)
data['Boat [Body]']=data['Boat [Body]'].apply(cleanup)
data['Age']=data['Age'].apply(pd.to_numeric, errors='coerce')
data = data[["Name","Age","Class/Dept","Boat [Body]"]]

data.head()


,Name,Age,Class/Dept,Boat [Body]
0,"AB SA'B, Mr Jirjis Y suf",26.0,3rd Class Passenger,[312]
1,"AB -AL-MUN , Mr N s f Q sim",27.0,3rd Class Passenger,15
2,"ABBING, Mr Anthony",42.0,3rd Class Passenger,
3,"ABBOTT, Mrs Rhoda Mary 'Rosa'",39.0,3rd Class Passenger,A
4,"ABBOTT, Mr Rossmore Edward",16.0,3rd Class Passenger,[190]


In [7]:
def checkPass(class_type):
    if "Passenger" in class_type:
        return "Passenger"
    else:
        return "Crew"
    
data["Crew/Pass"]=data["Class/Dept"].apply(checkPass)


In [8]:
def checkClass(class_type):
    if "Passenger" in class_type:
        return class_type.split(" ")[0]
    else:
        return "Crew"
    
data["Class"]=data["Class/Dept"].apply(checkClass)
data.head()


,Name,Age,Class/Dept,Boat [Body],Crew/Pass,Class
0,"AB SA'B, Mr Jirjis Y suf",26.0,3rd Class Passenger,[312],Passenger,3rd
1,"AB -AL-MUN , Mr N s f Q sim",27.0,3rd Class Passenger,15,Passenger,3rd
2,"ABBING, Mr Anthony",42.0,3rd Class Passenger,,Passenger,3rd
3,"ABBOTT, Mrs Rhoda Mary 'Rosa'",39.0,3rd Class Passenger,A,Passenger,3rd
4,"ABBOTT, Mr Rossmore Edward",16.0,3rd Class Passenger,[190],Passenger,3rd


In [9]:
def checkAdult(age):
    if age>=18:
        return "Adult"
    else:
        return "Child"

data["Adult/Child"]=data["Age"].apply(checkAdult)
data.head()

,Name,Age,Class/Dept,Boat [Body],Crew/Pass,Class,Adult/Child
0,"AB SA'B, Mr Jirjis Y suf",26.0,3rd Class Passenger,[312],Passenger,3rd,Adult
1,"AB -AL-MUN , Mr N s f Q sim",27.0,3rd Class Passenger,15,Passenger,3rd,Adult
2,"ABBING, Mr Anthony",42.0,3rd Class Passenger,,Passenger,3rd,Adult
3,"ABBOTT, Mrs Rhoda Mary 'Rosa'",39.0,3rd Class Passenger,A,Passenger,3rd,Adult
4,"ABBOTT, Mr Rossmore Edward",16.0,3rd Class Passenger,[190],Passenger,3rd,Child


In [10]:
def checkGender(name):
    firstname = name[name.index(",")+2:]
    salutation=firstname.split(" ")[0]
    #print salutation
    if salutation in ["Mr","Master"]:
        return "Male"
    else:
        return "Female"

data["Gender"]=data["Name"].apply(checkGender)
data.head()

,Name,Age,Class/Dept,Boat [Body],Crew/Pass,Class,Adult/Child,Gender
0,"AB SA'B, Mr Jirjis Y suf",26.0,3rd Class Passenger,[312],Passenger,3rd,Adult,Male
1,"AB -AL-MUN , Mr N s f Q sim",27.0,3rd Class Passenger,15,Passenger,3rd,Adult,Male
2,"ABBING, Mr Anthony",42.0,3rd Class Passenger,,Passenger,3rd,Adult,Male
3,"ABBOTT, Mrs Rhoda Mary 'Rosa'",39.0,3rd Class Passenger,A,Passenger,3rd,Adult,Female
4,"ABBOTT, Mr Rossmore Edward",16.0,3rd Class Passenger,[190],Passenger,3rd,Child,Male


In [11]:
def checkSurvival(boat):
    if boat.strip()=="" or "[" in boat:
        return 0
    else:
        return 1
data["Survival"]=data["Boat [Body]"].apply(checkSurvival)
data.head()

,Name,Age,Class/Dept,Boat [Body],Crew/Pass,Class,Adult/Child,Gender,Survival
0,"AB SA'B, Mr Jirjis Y suf",26.0,3rd Class Passenger,[312],Passenger,3rd,Adult,Male,0
1,"AB -AL-MUN , Mr N s f Q sim",27.0,3rd Class Passenger,15,Passenger,3rd,Adult,Male,1
2,"ABBING, Mr Anthony",42.0,3rd Class Passenger,,Passenger,3rd,Adult,Male,0
3,"ABBOTT, Mrs Rhoda Mary 'Rosa'",39.0,3rd Class Passenger,A,Passenger,3rd,Adult,Female,1
4,"ABBOTT, Mr Rossmore Edward",16.0,3rd Class Passenger,[190],Passenger,3rd,Child,Male,0


Summarization of features


In [14]:
data.groupby(["Crew/Pass"])["Survival"].sum()*100/data.groupby(["Crew/Pass"])["Survival"].count()

Crew/Pass
Passenger    36.75019
Name: Survival, dtype: float64